In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import sys
sys.path.insert(1,'D:\Github\DeepLake-Langchain')
import credentials
import os
OPENAI_API_KEY = credentials.openai
ACTIVE_LOOP_ORG_ID = credentials.active_loop_org_id
DATALAKE_KEY = credentials.active_loop
os.environ["ACTIVELOOP_TOKEN"] = DATALAKE_KEY


# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0,openai_api_key = OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key = OPENAI_API_KEY)

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVE_LOOP_ORG_ID
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


\

Dataset(path='hub://megatron17/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['7bef1d16-14dc-11ee-b60b-00d861dd19c7',
 '7bef1d17-14dc-11ee-98a7-00d861dd19c7']

In [7]:
docs

[Document(page_content='Napoleon Bonaparte was born in 15 August 1769', metadata={}),
 Document(page_content='Louis XIV was born in 5 September 1638', metadata={})]

In [8]:
# dir(DeepLake)

In [9]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [11]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


## Adding more data

In [12]:
#load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

#create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

#add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://megatron17/langchain_course_from_zero_to_hero already exists, loading from the storage


|

Dataset(path='hub://megatron17/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['10f5598d-14dd-11ee-b8f9-00d861dd19c7',
 '10f5598e-14dd-11ee-9aea-00d861dd19c7']

In [13]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0,openai_api_key = OPENAI_API_KEY)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [14]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


In [15]:
response = agent.run("When was Michael Jordan and Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out the birth dates of both people
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I need to find out when Michael Jordan was born
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer
Final Answer: Michael Jordan was born on 17 February 1963 and Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Michael Jordan was born on 17 February 1963 and Napoleon Bonaparte was born on 15 August 1769.
